In [1]:
# -*- coding: utf-8 -*-
import logging
import datetime
from logging.handlers import TimedRotatingFileHandler
from logging.handlers import RotatingFileHandler
from sklearn.externals import joblib
from collections import OrderedDict
import sys
from dayu.hooks.oss_hook import OSSHook

from dayu.hooks.hive_server_hook import HiveServerHook
from dayu.hooks.hive_cli_hook import HiveCliHook

import pandas as pd 
import numpy as np
from fastFM import sgd
from scipy import sparse
from scipy.sparse import csc_matrix
import time
from sklearn.metrics import auc,accuracy_score,roc_curve,roc_auc_score
from sklearn.metrics import confusion_matrix,recall_score
from sklearn import  metrics
from sklearn.linear_model import LogisticRegression

def split_table_name(datain):
    new_cols = []
    for column in datain.columns:
        if(len(column.split('.'))<2):
            return datain
        tb_name, col_name = column.split('.')
        new_cols.append((column, col_name))
    datain = datain.rename(columns=dict(new_cols))
    return datain

def read_from_hive2(output_file_name,insql,dtype):
    filename = output_file_name
    filepath = curr_dir+filename
    hive = HiveServerHook("warehouse_hive")
    hive.to_csv(insql,filepath , delimiter=',',lineterminator='\n', output_header=True)
    outdata = pd.read_csv(filepath, header=0,dtype=dtype)
    # 去除列名中带有的表名
    outdata = split_table_name(outdata)
    return outdata

curr_date = str(datetime.datetime.now())[0:10]
logging.info('当前日期: %s'%(curr_date))
curr_dir = '/home/souche/qiongjiu/hgc/'
curr_date=str(datetime.datetime.now())[0:10]
hive_cli = HiveCliHook("warehouse_hive")

DAYU_HOME : /home/souche/projects/datacenter-etl-v2


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


[2020-11-05 09:38:47,090] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/Grammar.txt
[2020-11-05 09:38:47,117] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/PatternGrammar.txt
[2020-11-05 09:38:48,561] {<ipython-input-1-b6909f377136>:54} INFO - 当前日期: 2020-11-05


In [2]:
#读取采购清洗完的数据
sql = """
SELECT purchase_order_id as car_id,
       brand_code,
       brand_name,
       series_code,
       series_name,
       model_code,
       model_name,
       cast(mileage/10000 AS DECIMAL(10,2)) AS mileage_std,
       color AS color_name,
       first_license_date as first_license_plate_date,
       province_name,
       city_name,
       cast(purchase_price/10000 AS DECIMAL(10,2)) AS purchase_price,
       date(pay_time) as order_create_time,
       age AS license_month,
       residual,
       -- 需要按照当年情况来修改
       datediff("2020-12-31 00:00:00", pay_time) AS days,
       cast(year(first_license_date) AS INT) AS license_year
FROM db_data.mid_car_dfc_purchase_order_flag_b
WHERE flag = 0 
and pay_time is not null
 -- 确认选取建模时间
and pay_time<="2020-11-02"
and province_name is not null
and city_name is not null
"""
dtype={'uid':str}
data= read_from_hive2('mid_car_dfc_purchase_order_flag',sql,dtype)


[2020-11-05 09:38:48,638] {hiveserver2:138} INFO - Using database default as default
[2020-11-05 09:38:48,808] {hive_server_hook:112} INFO - Running query: 
SELECT purchase_order_id as car_id,
       brand_code,
       brand_name,
       series_code,
       series_name,
       model_code,
       model_name,
       cast(mileage/10000 AS DECIMAL(10,2)) AS mileage_std,
       color AS color_name,
       first_license_date as first_license_plate_date,
       province_name,
       city_name,
       cast(purchase_price/10000 AS DECIMAL(10,2)) AS purchase_price,
       date(pay_time) as order_create_time,
       age AS license_month,
       residual,
       -- 需要按照当年情况来修改
       datediff("2020-12-31 00:00:00", pay_time) AS days,
       cast(year(first_license_date) AS INT) AS license_year
FROM db_data.mid_car_dfc_purchase_order_flag_b
WHERE flag = 0 
and pay_time is not null
 -- 确认选取建模时间
and pay_time<="2020-11-02"
and province_name is not null
and city_name is not null

[2020-11-05 09:38:49,8

[2020-11-05 09:39:57,933] {hive_server_hook:162} INFO - Written 870000 rows so far.
[2020-11-05 09:39:58,716] {hive_server_hook:162} INFO - Written 880000 rows so far.
[2020-11-05 09:39:59,501] {hive_server_hook:162} INFO - Written 890000 rows so far.
[2020-11-05 09:40:00,237] {hive_server_hook:162} INFO - Written 900000 rows so far.
[2020-11-05 09:40:01,044] {hive_server_hook:162} INFO - Written 910000 rows so far.
[2020-11-05 09:40:01,824] {hive_server_hook:162} INFO - Written 920000 rows so far.
[2020-11-05 09:40:02,601] {hive_server_hook:162} INFO - Written 930000 rows so far.
[2020-11-05 09:40:03,331] {hive_server_hook:162} INFO - Written 940000 rows so far.
[2020-11-05 09:40:04,162] {hive_server_hook:162} INFO - Written 950000 rows so far.
[2020-11-05 09:40:04,940] {hive_server_hook:162} INFO - Written 960000 rows so far.
[2020-11-05 09:40:05,722] {hive_server_hook:162} INFO - Written 970000 rows so far.
[2020-11-05 09:40:06,500] {hive_server_hook:162} INFO - Written 980000 rows 

In [3]:
data.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450163 entries, 0 to 1450162
Data columns (total 18 columns):
car_id                      1450163 non-null int64
brand_code                  1450059 non-null object
brand_name                  1450059 non-null object
series_code                 1450059 non-null object
series_name                 1450059 non-null object
model_code                  1450163 non-null object
model_name                  1450059 non-null object
mileage_std                 1450163 non-null float64
color_name                  1449920 non-null object
first_license_plate_date    1450163 non-null object
province_name               1450163 non-null object
city_name                   1450163 non-null object
purchase_price              1450163 non-null float64
order_create_time           1450163 non-null object
license_month               1450163 non-null int64
residual                    1450163 non-null float64
days                        1450163 non-null int64
lic

In [4]:
data['mileage_std'] = data['mileage_std'].astype('float')
data['purchase_price'] = data['purchase_price'].astype('float')
data['license_month'] = data['license_month'].astype('int')
data['residual'] = data['residual'].astype('float')

data.head()

,car_id,brand_code,brand_name,series_code,series_name,model_code,model_name,mileage_std,color_name,first_license_plate_date,province_name,city_name,purchase_price,order_create_time,license_month,residual,days,license_year
0,929372,brand-49,丰田,series-1072,汉兰达,054186468,2013款 汉兰达 2.7L 两驱7座探索版,7.20,4,2014-01-01,湖北,武汉,18.8,2018-03-30,51,0.571776,1007,2014
1,1027344,brand-49,丰田,series-1072,汉兰达,054186468,2013款 汉兰达 2.7L 两驱7座探索版,6.00,4,2014-03-01,黑龙江,哈尔滨,19.8,2018-05-26,51,0.602190,950,2014
2,2335083,brand-49,丰田,series-1072,汉兰达,054186468,2013款 汉兰达 2.7L 两驱7座探索版,8.00,4,2013-11-01,辽宁,大连,12.2,2020-10-10,84,0.371046,82,2013
3,63147,brand-49,丰田,series-1072,汉兰达,054186468,2013款 汉兰达 2.7L 两驱7座探索版,8.35,4,2013-11-01,山东,济南,18.8,2016-07-30,33,0.571776,1615,2013
4,662674,brand-49,丰田,series-1072,汉兰达,054186468,2013款 汉兰达 2.7L 两驱7座探索版,5.00,3,2014-09-01,黑龙江,哈尔滨,22.0,2017-09-23,37,0.669100,1195,2014


In [5]:
# 读取可估价的车型 224239
model = pd.read_csv(curr_dir+"2020-11-02版车型参数及独热编码.csv", header=0, low_memory=False)
model = model[['model_code', 'model_year', 'new_car_price','year_1', 'year_2', 'year_3', 'year_4', 'year_5', 'year_6',
                     'year_7', 'year_8', 'year_9', 'year_10', 'year_11', 'year_12', 'year_13', 
                     'year_14', 'year_15', 'year_16','rate','rate_count']]
print("合并前数据数目为:%d, 车型数量为：%d" % (data.shape[0], len(data['model_code'].unique())))
data['model_code'] = data['model_code'].astype('str')
data = pd.merge(data, model, on='model_code', how='inner')
print("<---------------------------------------->")
print("合并后数据数目为:%d, 车型数量为：%d" % (data.shape[0], len(data['model_code'].unique())))

合并前数据数目为:1450163, 车型数量为：27744
<---------------------------------------->
合并后数据数目为:1446922, 车型数量为：27064


In [6]:
#去掉一些不合理的数据 meimei
data_m = data[(data['mileage_std']>=0.01) & (data['mileage_std']<=50)]
data_m = data_m[(data_m['purchase_price']>=0.2) & (data_m['purchase_price']<=300)]
data_m = data_m[(data_m['license_month']>=1) & (data_m['license_month']<=180)]
data_m = data_m[(data_m['residual']>=0.02) & (data_m['residual']<1.2)]
data_m = data_m[(data_m['model_year']>=2006)]
data_m = data_m[data_m['license_year']>=2005]
data_m = data_m[(data_m['new_car_price']>=0.85) & (data_m['new_car_price']<=500)]

#需要计算
data_m['per_mile'] = round(data_m['mileage_std']/(data_m['license_month']/12),2)

#需要计算
data_m['year_err'] = data_m['license_year'] - data_m['model_year']
data_m = data_m[(data_m['year_err']>=-2) & (data_m['year_err']<=6)]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))

增加限制后，数据数量为:1399647, 车型数量为:25303


In [7]:
# 读取编码完成的省份数据
province = pd.read_csv("province_new.csv", header=0, low_memory=False)
data2m = pd.merge(data_m, province, how='left', on='province_name')
data2m.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1399647 entries, 0 to 1399646
Data columns (total 71 columns):
car_id                      1399647 non-null int64
brand_code                  1399543 non-null object
brand_name                  1399543 non-null object
series_code                 1399543 non-null object
series_name                 1399543 non-null object
model_code                  1399647 non-null object
model_name                  1399543 non-null object
mileage_std                 1399647 non-null float64
color_name                  1399405 non-null object
first_license_plate_date    1399647 non-null object
province_name               1399647 non-null object
city_name                   1399647 non-null object
purchase_price              1399647 non-null float64
order_create_time           1399647 non-null object
license_month               1399647 non-null int64
residual                    1399647 non-null float64
days                        1399647 non-null int64
lic

In [8]:
# 该函数为计算保值率 
def computer_with_license_month(tar):
    license_month = tar['license_month']
    if(license_month <= 12):
        return tar['year_1']
    else:
        year = license_month//12
        #当前年保值率
        keep_max = tar["year_"+str(int(year))]
        #下一年的保值率
        keep_min = tar["year_"+str(int(year+1))]
        
        #相比于上一年，已经过了几个月
        mon = license_month - 12 * year
        tem = (keep_max-keep_min)/12
        
        return round(keep_max - tem*mon, 4)
    #return tar
data2m['keep_value'] = data2m[['license_month','year_1', 'year_2', 'year_3', 'year_4','year_5', 'year_6', 'year_7', 'year_8', 'year_9', 'year_10', 'year_11','year_12', 'year_13', 'year_14', 'year_15', 'year_16']].to_dict(orient='records')
print("====")
data2m['keep_value'] = data2m['keep_value'].map(lambda tar:computer_with_license_month(tar))
#data2m = data2m.apply(computer_with_license_month, axis=1)

====


In [9]:
import pandas as pd
quality = pd.read_csv(curr_dir+"2020-11-02版车型参数及独热编码.csv", header=0, low_memory=False)


In [10]:
data3m = data2m.drop(['year_1', 'year_2', 'year_3', 'year_4', 
                    'year_5', 'year_6', 'year_7', 'year_8', 
                    'year_9', 'year_10', 'year_11', 'year_12', 
                    'year_13', 'year_14', 'year_15', 'year_16'], axis=1)

quality = pd.read_csv(curr_dir+"2020-11-02版车型参数及独热编码.csv", header=0, low_memory=False)
quality = quality[['model_code', 'quality_mile', 'quality_year']]
print(quality.shape)

data3m = pd.merge(data3m, quality, how='left', on='model_code')

def get_quality(license_month,mileage_std,quality_mile,quality_year):
    year = round(license_month/12, 2)
    if mileage_std < quality_mile and year< quality_year:
        return 1
    else:
        return 0
       
data3m['quality'] = list(map(lambda license_month,mileage_std,quality_mile,quality_year:get_quality(license_month,mileage_std,quality_mile,quality_year),
          data3m['license_month'],data3m['mileage_std'],data3m['quality_mile'],data3m['quality_year']))
data3m = data3m.drop(['quality_mile', 'quality_year'], axis=1)


(65250, 3)


In [11]:
#计算一下log
# def feature_log(tar):
#     tar['mile_log'] = round(math.log(tar['mileage_std'], 2), 4)
#     tar['license_month_log'] = round(math.log(tar['license_month'], 2), 4)
#     tar['new_car_price_log'] = round(math.log(tar['new_car_price'], 2), 4)
#     return tar
#data3m = data3m.apply(feature_log, axis=1)
# 切换使用 np.log2 效率高一些 
data3m['mile_log']=round(np.log2(data3m['mileage_std']), 4)
data3m['license_month_log']=round(np.log2(data3m['license_month']), 4)
# data3m['license_month_log']=data3m['license_month']
data3m['new_car_price_log']=round(np.log2(data3m['new_car_price']), 4)

In [12]:
# 数据筛选  查看每个车型对应的建模数据
data_counts_mei = data3m.groupby('model_code')['car_id'].count().rename('count').reset_index()
data5m = pd.merge(data3m, data_counts_mei, how='left', on='model_code')
data5m = data5m.reset_index(drop=True)
data5m.to_csv(curr_dir+"2020-11-02版带count处理完成的采购数据m.csv", header=True, index=False)

### 按照model_code划分数据集

In [14]:
# 之前测试了两种 数据，所以有两条记录。 

#交易条数超过5条
datahm = data5m[data5m['count']>=5]
#交易条数少于5条
datalm = data5m[data5m['count']<5]

print("交易条数超过5条的数据量是:%d, 车型数量是:%d" % (datahm.shape[0], len(datahm['model_code'].unique())))
print("交易条数少于5条的数据量是:%d, 车型数量是:%d" % (datalm.shape[0], len(datalm['model_code'].unique())))

交易条数超过5条的数据量是:1381029, 车型数量是:16706
交易条数少于5条的数据量是:18618, 车型数量是:8597


In [15]:
"""
单个model_code, 交易记录数超过5条, 使用
StratifiedShuffleSplit来进行划分, 测试训练
比例2:8; 单个model_code交易记录数少于5条, 
手动划分
"""
#step1:首先划分数据量超过5条的
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold

stratified = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

# 切meimei 的数据
for sub_set1, sub_set2 in stratified.split(datahm, datahm['model_code']):
    train_1m = datahm.iloc[sub_set1]
    test_1m = datahm.iloc[sub_set2]
print(train_1m.shape)
print(test_1m.shape)

(1104823, 61)
(276206, 61)


In [18]:
datalm['car_id'] = datalm['car_id'].map(lambda x:str(x))

/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
#按对应比例划分训练集和测试集
from sklearn.utils import shuffle
def split_data(data):
    model_code_list = list(data['model_code'].unique())
    #用来保存抽取的训练数据
    data_train = data[data['car_id']=="192044"]
    #用来保存抽取的测试数据
    data_test = data[data['car_id']=="1112363"]
    for i in range(len(model_code_list)):
        model_code = model_code_list[i]
        if model_code=="12365-n":
            continue
        data_tem = data[data['model_code']==model_code]
        if(data_tem.shape[0]==1):
            data_train = pd.concat([data_train, data_tem])
        elif(data_tem.shape[0]==2):
            data_tem_test = data_tem.iloc[0:1, :]
            data_tem_train = data_tem.iloc[1:2, :]
            data_test = pd.concat([data_test, data_tem_test])
            data_train = pd.concat([data_train, data_tem_train])
        #三条以上的数据才能划分训练集和测试集
        else:
            data_tem = shuffle(data_tem)
            n = int(round(data_tem.shape[0]*0.2))
            data_tem_test = data_tem.iloc[0:n, :]
            data_tem_train = data_tem.iloc[n:, :]
            
            data_test = pd.concat([data_test, data_tem_test])
            data_train = pd.concat([data_train, data_tem_train])
        if(i%1000==0):
            print("已完成%d个model_code数据集划分, 共%d个, 进度%.2f%%" % 
                  (i, len(model_code_list), i/len(model_code_list)*100))
    return data_train, data_test

train_2m, test_2m = split_data(datalm)


已完成0个model_code数据集划分, 共8597个, 进度0.00%
已完成1000个model_code数据集划分, 共8597个, 进度11.63%
已完成2000个model_code数据集划分, 共8597个, 进度23.26%
已完成3000个model_code数据集划分, 共8597个, 进度34.90%
已完成4000个model_code数据集划分, 共8597个, 进度46.53%
已完成5000个model_code数据集划分, 共8597个, 进度58.16%
已完成6000个model_code数据集划分, 共8597个, 进度69.79%
已完成7000个model_code数据集划分, 共8597个, 进度81.42%
已完成8000个model_code数据集划分, 共8597个, 进度93.06%


In [20]:
print(train_2m.shape)
print(test_2m.shape)

(12943, 61)
(5675, 61)


In [21]:

data_trainm = pd.concat([train_1m, train_2m])
data_trainm = data_trainm.reset_index(drop=True)

data_testm = pd.concat([test_1m, test_2m])
data_testm = data_testm.reset_index(drop=True)

In [ ]:
data_trainm.to_csv(curr_dir+"2020-11-02版采购数据data_train2.csv", header=True, index=False)
data_testm.to_csv(curr_dir+"2020-11-02版采购数据data_test2.csv", header=True, index=False)